In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
img_color = cv2.imread('test_imgs/color_new.png')
img_depth = cv2.imread('test_imgs/depth_new.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (480, 640, 3)
depth shape:  (480, 640)


In [3]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [4]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [5]:
# RED
lower_red1 = np.array([0, 120, 50])
upper_red1 = np.array([20, 255, 255])
lower_red2 = np.array([160,120,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 75, 100])
upper_pink1 = np.array([10, 120, 255])
lower_pink2 = np.array([150,75,100])
upper_pink2 = np.array([179,120,255])

# GREEN
lower_green = (70-20, 60, 60)
upper_green = (70+20, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+10, 255, 150)

# VIOLET
lower_violet = (130-20, 30, 30)
upper_violet = (130+20, 255, 255)

In [6]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    if color == 'red':
        cv2.imshow('src_bin', src_bin)
        cv2.imshow(f'{color}_filtered', each_color_filtered)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 250:
            continue
        
        if color == 'red' and i==1:
            cv2.imshow('blk clr', block_color)
            # cv2.imwrite('./test_imgs/red_block1_rgb.png', block_color)
            cv2.imshow('blk msk', block_mask)
            # cv2.imwrite('./test_imgs/red_block1_mask.png', block_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [7]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [8]:
cv2.imshow('tower mask', tower_mask)
# cv2.imwrite('./test_imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
# cv2.imwrite('./test_imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
len(blocks_rgb_blue)

6

In [10]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b))

green
6
pink
6
yellow
6
blue
6
violet
6
red
6


In [11]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[322.282420, 0, 320.818268],[0, 322.282420, 178.779297],[0, 0, 1]]

In [12]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [13]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [14]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [15]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 34089 points.
[[-2.34823723e-05 -1.22886669e-04  4.78431379e-04]
 [-2.19978626e-05 -1.22886669e-04  4.78431379e-04]
 [-2.05133529e-05 -1.22886669e-04  4.78431379e-04]
 ...
 [ 7.59722067e-06  2.33777307e-04  3.96078423e-04]
 [ 8.82619998e-06  2.33777307e-04  3.96078423e-04]
 [ 1.00551793e-05  2.33777307e-04  3.96078423e-04]]


In [16]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(64, 0.00001)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        if color=='red':
            o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [17]:
# o3d.visualization.draw_geometries(all_pcd)

In [18]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
o3d.visualization.draw_geometries([pcd_combined])

PointCloud with 32938 points.


In [19]:
mesh = o3d.io.read_triangle_mesh("mesh/jenga_tower_side.stl")
print(mesh)

TriangleMesh with 12 points and 4 triangles.


In [20]:
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

In [21]:
pcd_target = mesh.sample_points_uniformly(number_of_points=32937)
o3d.visualization.draw_geometries([pcd_target])

In [22]:
import copy

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [23]:
source = pcd_combined
target = pcd_target
threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)

In [24]:
o3d.visualization.draw_geometries([source, target])

In [25]:
source_temp2 = copy.deepcopy(source)
target_temp2 = copy.deepcopy(target)

In [26]:
type(source_temp2.points)

open3d.cpu.pybind.utility.Vector3dVector

In [27]:
a = o3d.cpu.pybind.utility.Vector3dVector(np.array(source.points)*500000)

In [28]:
source_temp2.points = a

In [29]:
np.array(target.points[0])

array([ 37.5       , 110.64289561, -25.37004399])

In [30]:
np.array(source_temp2.points[0])

array([ -9.43144444, -48.75988296, 192.15686189])

In [31]:
o3d.visualization.draw_geometries([source_temp2, target])

In [32]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [33]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")
    source = copy.deepcopy(source_temp2)
    target = copy.deepcopy(pcd_target)
    trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                             [0.0, 0.0, -1.0, 0.0], [0.0, 0.0, 0.0, -1.0]])
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [34]:
voxel_size = 0.000005  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.000005.
:: Estimate normal with search radius 0.000010.
:: Compute FPFH feature with search radius 0.000025.
:: Downsample with a voxel size 0.000005.
:: Estimate normal with search radius 0.000010.
:: Compute FPFH feature with search radius 0.000025.


In [35]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %f," % voxel_size)
    print("   we use a liberal distance threshold %f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [36]:
result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)
print(result_ransac)
draw_registration_result(source_down, target_down, result_ransac.transformation)

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.000005,
   we use a liberal distance threshold 0.000008.


RegistrationResult with fitness=0.000000e+00, inlier_rmse=0.000000e+00, and correspondence_set size of 0
Access transformation to get result.
